In [1]:
# This code to incorprate dropout reglrsn into model5
    # - how to incorporate dropouit into multiple parts of an LLM
    # - how to toggle dropout in attention algo for evaluation
    # - imp of training set variability for training performance


In [2]:
# Few cells below to show Fineweb dataset and how to download them

In [2]:
from datatrove.pipeline.readers import ParquetReader  # to download the Fineweb dataset

/Users/raeez/.pyenv/versions/jupyter-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# how many doc to retrive, each doc has ~750 tokens
numDocs= 10

# import the docs
data_reader = ParquetReader('hf://datasets/HuggingFaceFW/fineweb/data',limit=numDocs)

In [5]:
# get one dataset doc
text = next(iter(data_reader()))
dir(text)

2026-02-03 15:22:41.924 | INFO     | datatrove.pipeline.readers.base:read_files_shard:203 - Reading input file CC-MAIN-2013-20/000_00000.parquet, 1/27468


['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'id',
 'media',
 'metadata',
 'text']

In [6]:
text.metadata

{'dump': 'CC-MAIN-2013-20',
 'url': 'http://%20jwashington@ap.org/Content/Press-Release/2012/How-AP-reported-in-all-formats-from-tornado-stricken-regions',
 'date': '2013-05-18T05:48:54Z',
 'file_path': 's3://commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/warc/CC-MAIN-20130516092621-00000-ip-10-60-113-184.ec2.internal.warc.gz',
 'language': 'en',
 'language_score': 0.9721424579620361,
 'token_count': 717}

In [7]:
text.text

"How AP reported in all formats from tornado-stricken regionsMarch 8, 2012\nWhen the first serious bout of tornadoes of 2012 blew through middle America in the middle of the night, they touched down in places hours from any AP bureau. Our closest video journalist was Chicago-based Robert Ray, who dropped his plans to travel to Georgia for Super Tuesday, booked several flights to the cities closest to the strikes and headed for the airport. He’d decide once there which flight to take.\nHe never got on board a plane. Instead, he ended up driving toward Harrisburg, Ill., where initial reports suggested a town was destroyed. That decision turned out to be a lucky break for the AP. Twice.\nRay was among the first journalists to arrive and he confirmed those reports -- in all formats. He shot powerful video, put victims on the phone with AP Radio and played back sound to an editor who transcribed the interviews and put the material on text wires. He then walked around the devastation with th

In [3]:
import numpy as np
import matplotlib.pyplot as plt

import math
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
n_vocab = tokenizer.vocab_size

In [5]:
# hyper para for GPT2-124M
embed_dim = 768 #embedding dim
seq_len = 256 #max seq len
n_heads = 12 # attention heads
n_blocks = 12 # tranformer blocks
#each transformer block has 12 atention heads
batch_size = 8
dropout = .1 #n dropoout proportion(10% dropout)


device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [6]:
n_vocab

50257

Create a fineweb dataset

In [7]:
numDocs = 1000 # each doc has ~750 tokens
data_reader = ParquetReader('hf://datasets/HuggingFaceFW/fineweb/data',limit=numDocs)

In [8]:
# join all texts into one token vector
texttokens = np.array([])
for t in data_reader():
    texttokens = np.append(texttokens, tokenizer.encode(t.text))

# need a pytorch tensor for trianing
# texttokens = to

2026-02-03 15:52:48.720 | INFO     | datatrove.pipeline.readers.base:read_files_shard:203 - Reading input file CC-MAIN-2013-20/000_00000.parquet, 1/27468
Token indices sequence length is longer than the specified maximum sequence length for this model (1848 > 1024). Running this sequence through the model will result in indexing errors


In [9]:
print(type(texttokens))
print(texttokens.shape)

<class 'numpy.ndarray'>
(748751,)


In [10]:
#need a pytorch tensor for trianing
texttokens = torch.tensor(texttokens, dtype=torch.long)

#set() doesnt work on torch tensors
print(f'\nTher are {len(texttokens):,} tokens, of which {len(set(texttokens.numpy())):,} are unique')


Ther are 748,751 tokens, of which 34,895 are unique


function to get training/testing data

In [11]:
# train/test split
train_ratio = .9

#index to split data
test_split_point = int(train_ratio * len(texttokens))

train_data = texttokens[:test_split_point]
test_data = texttokens[test_split_point:]
# in this split 10% of the last part is training set, whi9ch is not a good split
# ideally we would want a random split


# a func that return a batch of data samoples
def get_data_batch(training = True):
    # pick the dataset to use
    if training:
        data = train_data
    else:
        data = test_data

    #pick random idices to start
    ix = torch.randint(len(data) - seq_len, size = (batch_size,))

    #get the data and targets (via broadcasting outer product)
    X = data[ix[:,None] + torch.arange(seq_len)]  # now this is becomes a matrix
    y = data[ix[:,None] + torch.arange(1,seq_len+1)]

    return X,y

#example 
X,y = get_data_batch()
print(f'Input data (size {X.shape}):\n',X)
print(f'\n\nTargets (size {y.shape}):\n',y)

Input data (size torch.Size([8, 256])):
 tensor([[ 9911,   503,   379,  ..., 40709, 38622,   318],
        [ 5793,   373,  4615,  ...,   257,  3644,  1080],
        [ 8128,   340,   373,  ...,   284,   262, 19154],
        ...,
        [ 3015,   284,  3051,  ...,  5318,   287,  5575],
        [ 3607,   262,  5852,  ...,  9690,   284,   262],
        [  714,   307,  2233,  ...,  8607, 25335,  3961]])


Targets (size torch.Size([8, 256])):
 tensor([[  503,   379,   399,  ..., 38622,   318, 10488],
        [  373,  4615,    11,  ...,  3644,  1080,  1900],
        [  340,   373, 10403,  ...,   262, 19154,   338],
        ...,
        [  284,  3051,   287,  ...,   287,  5575, 48023],
        [  262,  5852,  7353,  ...,   284,   262,  7059],
        [  307,  2233,   284,  ..., 25335,  3961,   287]])


In [12]:
ix = torch.randint(len(train_data) - seq_len, size = (batch_size,))

In [13]:
ix

tensor([ 97305, 307344, 535317, 503209, 639054, 254907, 375950, 155301])

In [14]:
ix[:,None] + torch.arange(seq_len)

tensor([[ 97305,  97306,  97307,  ...,  97558,  97559,  97560],
        [307344, 307345, 307346,  ..., 307597, 307598, 307599],
        [535317, 535318, 535319,  ..., 535570, 535571, 535572],
        ...,
        [254907, 254908, 254909,  ..., 255160, 255161, 255162],
        [375950, 375951, 375952,  ..., 376203, 376204, 376205],
        [155301, 155302, 155303,  ..., 155554, 155555, 155556]])

In [15]:
ix[:,None] + torch.arange(1,seq_len+1)

tensor([[ 97306,  97307,  97308,  ...,  97559,  97560,  97561],
        [307345, 307346, 307347,  ..., 307598, 307599, 307600],
        [535318, 535319, 535320,  ..., 535571, 535572, 535573],
        ...,
        [254908, 254909, 254910,  ..., 255161, 255162, 255163],
        [375951, 375952, 375953,  ..., 376204, 376205, 376206],
        [155302, 155303, 155304,  ..., 155555, 155556, 155557]])

In [16]:
train_data[ix[:,None] + torch.arange(seq_len)]

tensor([[  262,  4302,  1204,  ...,   995,   286,   262],
        [  287,  2769, 10150,  ...,   481,   307,  2683],
        [   13,   198,  3237,  ...,   284,  1064,   262],
        ...,
        [  284, 14658,   262,  ...,    11, 19010,   447],
        [ 3521,   447,   247,  ...,   286,  1162,  2817],
        [  257,  1474,    12,  ..., 24470,  5660,  2364]])

In [17]:
train_data[ix[:,None] + torch.arange(1,seq_len+1)]

tensor([[ 4302,  1204, 35445,  ...,   286,   262, 30477],
        [ 2769, 10150,   329,  ...,   307,  2683,   546],
        [  198,  3237,   286,  ...,  1064,   262, 29262],
        ...,
        [14658,   262,  5531,  ..., 19010,   447,   247],
        [  447,   247,    83,  ...,  1162,  2817, 12023],
        [ 1474,    12, 25833,  ...,  5660,  2364, 30061]])

The model with dropout

In [18]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        
        super().__init__()
    
        #head dimensionality is embed_dim split across the heads
        self.num_heads = n_heads
        self.head_dim = embed_dim // n_heads
    
        # the three Q,K,V weight matrices are init as one, and are split inside attention eqn
        self.QKV = nn.Linear(embed_dim, 3*embed_dim, bias=True)
    
        #final linear projection merges the heads outputs
        self.W0 = nn.Linear(embed_dim, embed_dim, bias=True)
    
    def forward(self, x):
        # extract the dimension size of the inputs(token embedds)
        B, T, E = x.shape # [batch, tokens (or seq_len), embed_dim]
        

        #push data through Q,K and V in one concatenated matrix
        qkv = self.QKV(x) #[batch, seq_len, 3*embed]
        q,k,v = torch.split(qkv, E, dim=2) # each matrix is [B,T,E]

        # reshape to [B,T,nHeads, head_dim]
        # and then transpose to [B, nHeads, T, head_dim]
        q = q.view(B, T, self.num_heads, self.head_dim).transpose(1,2) #[B, num_heads, T, head_dim]
        k = k.view(B, T, self.num_heads, self.head_dim).transpose(1,2)
        v = v.view(B, T, self.num_heads, self.head_dim).transpose(1,2)

        # Pytorchs SDPA func handles multi head shapes

        # IMP-----> dropout enable for training
        # self.training is an attribute inherited from nn.Module()
        # print("B,T,E =", B, T, E)
        # print("q,k,v shapes:", q.shape, k.shape, v.shape)
        dropp = dropout if self.training == True else 0 
        out = F.scaled_dot_product_attention(q,k,v,is_causal=True, dropout_p = dropp)

        # recombine heads : (B,nHeads,T,head_dim) -> [B,T,E]
        out = out.transpose(1,2).reshape(B,T,E)
    

        #finallt apply linear mixing matrix
        out = self.W0(out)

        return out

class TransformerBlock(nn.Module):
    def __init__(self):
        super().__init__()

        #attention subblock
        self.layernorm_1 = nn.LayerNorm(embed_dim,eps=1e-5)
        self.attn = MultiHeadAttention()

        #feedfwd (MLP) sublayer
        self.layernorm_2 = nn.LayerNorm(embed_dim,eps=1e-5)
        self.mlp_1 = nn.Linear(embed_dim,4*embed_dim,bias=True) # 4x expansion
        self.gelu = nn.GELU()
        self.mlp_2 = nn.Linear(4*embed_dim, embed_dim, bias=True) #4x contraction

        self.trn_dropout = nn.Dropout(dropout)  # IMP--> Dropout here

    def forward(self,x):

        ## ----attention sublayer ------##
        x_att = self.layernorm_1(x) # pre attn normalisn
        # IMP ---> Dropout below
        x_att = x + self.trn_dropout(self.attn(x_att)) # run through attention, then add pre attn activations
        # attention -> dropout-> add residual
        
        #MLP
        x_ff = self.layernorm_2(x_att) # pre MLP normlsn
        x_ff = self.mlp_2( self.gelu( self.mlp_1(x_ff))) # expansion-contraction
        x_ff = x_att + self.trn_dropout(x_ff) # dropout the MLP and add back to embedd
        
        
        return x_ff
        
        

In [19]:
MultiHeadAttention().training

True

In [20]:
class LanguageModel(nn.Module):
    def __init__(self):
        super().__init__()

        # token + posn embedds
        self.wte = nn.Embedding(n_vocab, embed_dim) # token embedds
        self.wpe = nn.Embedding(seq_len, embed_dim) # posn embedds
        # IMP----> Dropout here
        self.emb_dropout = nn.Dropout(dropout)
        
        #n mutliple Transformer blocks
        self.transformerBlocks = nn.Sequential(*[TransformerBlock() for _ in range(n_blocks)])

        # embedding to output (linear) layer
        self.layernorm_final = nn.LayerNorm(embed_dim,eps=1e-5) # final layernorm after all txf blocks
        #unembed matirx
        self.final_head = nn.Linear(embed_dim, n_vocab, bias=False)
        #final ouput layer (unembedd) tied to token embedd
        self.final_head.weight = nn.Parameter(self.wte.weight)


        self.apply(self.weightInits) #apply the input func (weightInits) iteratively to all elements of this class
    
    def weightInits(self, module):
        
        # init nn.Linear to normal with std=.02
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0, std=.02)

            # init the bias terms to zero
            if module.bias is not None:
                nn.init.zeros_(module.bias)

        # Init nn.Embeddings to Xavier
        if isinstance(module, nn.Embedding):
            nn.init.xavier_normal_(module.weight)
                
    def forward(self, idx):

        #----------embeddings-------------##
        token_emb = self.wte(idx)  # [B,T,E]   T is seq_len and E is embed_dim
        posit_emb = self.wpe(torch.arange(idx.shape[-1],device=device)) #[seq_len, embed_dim]
        x = token_emb + posit_emb #[B,T,E]
        x = self.emb_dropout(x) # IMP----> DROPOUT after summing E+P
        ##--------------------------------##

        #n
        ##--pass through each transformer blocks----##
        x = self.transformerBlocks(x)
        ##-------------------------##

        #-----finally unembeddings----##
        x = self.layernorm_final(x)
        logits = self.final_head(x) # [B,T, n_vocab]

        #   IMP-----> we output logists here, no logsoftmax
        outputs = logits/math.sqrt(embed_dim)
        return outputs
        # outputs is [batch, seq_len, n_vocab]

    def generate(self,idx,temperature=1.,max_new_tokens=50):
        for _ in range(max_new_tokens):
            # fwd passb
            logits = self(idx[:,-seq_len:]) # [B,T,n_vocab]   get preds, but only from past seq_len tokens 
            logits = logits[:,-1,:] #[B,n_vocab]   extract last tokens logitsto predict the next

            # apply softmax with temp to get prob values over all tokens in vocab - with temp
            probs = F.softmax(logits/temperature,dim=-1) #[B,n_vocab]

            #probabilistically sample next token from distbn
            idx_next = torch.multinomial(probs, num_samples=1) # [batch,1]
            
            #append 
            idx = torch.cat((idx, idx_next),dim=1) #[batch, (tokens+1)]
        return idx

In [21]:
model = LanguageModel().to(device)

TRAIN on final token

In [22]:
# create loss and omptimizer funcitons
loss_function = nn.NLLLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=.001, weight_decay=.01)  

In [23]:
# check loss func with sizes
X,y = get_data_batch()
logits = model(X.to(device))

# calculate losses on final target token (with logsoftmax
# NOTE WE USED logsoftmax on logits in loss_func
loss  = loss_function(F.log_softmax(logits[:,-1,:],dim=-1),y[:,-1].to(device))
print('\nLoss:', loss)


Loss: tensor(10.8214, device='mps:0', grad_fn=<NllLossBackward0>)


In [24]:
print(y)
print(y[:,-1])

tensor([[  345,  2193,  1141,  ...,   901,    13,  1114],
        [ 4271,  1900,   357,  ...,   250, 15645, 16929],
        [   12, 31160,   198,  ...,    13,  1115,  9210],
        ...,
        [13838,    25, 21523,  ...,    17,  7133,   290],
        [   25,  6182,    12,  ...,    12,  9413,  2214],
        [ 9533,  6612,  3963,  ...,   683,  3830,   257]])
tensor([ 1114, 16929,  9210,   340,   307,   290,  2214,   257])


Now train the model!!!!

In [ ]:
num_samples = 1234

#init the loss
train_loss = []
test_loss = []


for sampli in range(num_samples):
    # get batch of data
    X,y = get_data_batch()
    
    model.zero_grad(set_to_none=True)

    #fwd pass
    logits = model(X.to(device))
    loss = loss_function(F.log_softmax(logits[:,-1,:],-1),y[:,-1].to(device))

    #bckprop
    loss.backward()
    optimizer.step()

    # sum the batch loss
    train_loss.append(loss.item())

    if sampli%80==0:    
        with torch.no_grad():
            model.eval() # IMP-----> deactivate dropout during testing and eval
            # NOTE: THe above will only deacticate dropout() class instances, not dropout using F.dropout function
            # byut model.eval() sets self.training=False for MultiHeadAttention class
            X,y = get_data_batch(False) # False -> testset data
            out = model(X.to(device))
            thisloss = loss_function(F.log_softmax(out[:,-1,:],-1),y[:,-1].to(device))
            test_loss.append(thisloss.item())
            model.train() # reactivate dropout
            
            print(f'Sample {sampli} train loss: {train_loss[-1]}, test loss:m{test_loss[-1]}')

Sample 0 train loss: 10.824453353881836, test loss:m10.806079864501953
Sample 80 train loss: 9.439631462097168, test loss:m8.874025344848633
